In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load CSV
df = pd.read_csv("/content/drive/MyDrive/mood/combined.csv")

# Separate features and labels
X = df.drop("Emotion", axis=1)
y = df["Emotion"]

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Build MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train on the full dataset
model.fit(X_scaled, y_encoded, epochs=50, batch_size=32)

# Save the trained model
model.save("/content/drive/MyDrive/mood/emotiondetector_mlp_model.h5")

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1840 - loss: 2.0280
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2358 - loss: 1.9212
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2917 - loss: 1.8143
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3024 - loss: 1.8231
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3164 - loss: 1.7818
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3249 - loss: 1.7468
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3076 - loss: 1.7577
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3157 - loss: 1.7206
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3361 - loss: 1.7226
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3065 - loss: 1.7291
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3504 - loss: 1.7062
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3286 - loss: 1.7080


In [4]:
print("Expected feature order:")
print(list(X.columns))

Expected feature order:
['Mean_Pitch', 'Var_Pitch', 'Max_Pitch', 'Min_Pitch', 'Mean_Energy', 'Var_Energy', 'Max_Energy', 'Min_Energy']


In [6]:
import joblib
from tensorflow.keras.models import load_model

# Save scaler
joblib.dump(scaler, "/content/drive/MyDrive/mood/emotiondetectornlw_scaler.pkl")

# Save label encoder
joblib.dump(le, "/content/drive/MyDrive/mood/emotiondetectornlw_labelencoder.pkl")


['/content/drive/MyDrive/mood/emotiondetectornlw_labelencoder.pkl']

Final FULL CODE

In [1]:
import librosa
import joblib
import sounddevice as sd
from scipy.io.wavfile import write, read

# Sampling rate
sr = 22050
fs = 44100  # for recording

# to Load Models and Preprocessors
# Load MLP model
mlp_model = load_model("emotiondetector_mlp_model.h5")
scaler_mlp = joblib.load("emotion_scaler.pkl")
le_mlp = joblib.load("emotion_labelencoder.pkl")

# Load KNN model
knn_model = joblib.load("knn_emotion_model.pkl")
le_knn = joblib.load("emotion_labelencoder.pkl")  # same encoder used for both

# to record Audio
def record_audio(filename='output.wav', seconds=3):
    print(f"\n Recording for {seconds} seconds...")
    audio = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
    sd.wait()
    write(filename, fs, audio)
    print(f"Recording saved as {filename}")

def play_audio(filename='output.wav'):
    fs, data = read(filename)
    print(f"\n Playing back {filename}...")
    sd.play(data, fs)
    sd.wait()

# to extract Features
def load_fixed_duration(file_path, sr, duration):
    y, _ = librosa.load(file_path, sr=sr, duration=duration)
    target_length = int(sr * duration)
    y = librosa.util.fix_length(y, size=target_length)
    return y

def split_audio_pitch(file_path, sr):
    o_audio = load_fixed_duration(file_path, sr, 3.0)
    Nw = int(sr * 0.6)
    f0 = librosa.yin(o_audio, fmin=50, fmax=8000, sr=sr, frame_length=Nw, hop_length=int(Nw/2), center=False)
    frames = librosa.util.frame(o_audio, frame_length=Nw, hop_length=int(Nw/2))
    energy = np.sqrt(np.mean(frames**2, axis=0))
    return f0, energy

def ML_feed(f0, energy):
    feed = [
        np.mean(f0), np.var(f0), np.max(f0), np.min(f0),
        np.mean(energy), np.var(energy), np.max(energy), np.min(energy)
    ]
    return np.array(feed).reshape(1, -1)

#  Prediction Logic
def predict_emotion(model_type, features):
    if model_type == "mlp":
        scaled = scaler_mlp.transform(features)
        probs = mlp_model.predict(scaled)
        pred = np.argmax(probs, axis=1)
        emotion = le_mlp.inverse_transform(pred)[0]
    elif model_type == "knn":
        pred = knn_model.predict(features)
        emotion = le_knn.inverse_transform(pred)[0]
    else:
        raise ValueError("Unknown model type.")
    return emotion

# Main Execution
if __name__ == "__main__":
    print(" Welcome to Emotion Detector")
    record_audio("output.wav", seconds=3)
    play_audio("output.wav")

    # Ask user to choose model
    print("\n Choose model for prediction:")
    print("1 - MLP Neural Network")
    print("2 - KNN Classifier")
    choice = input("Enter 1 or 2: ").strip()

    if choice == "1":
        model_type = "mlp"
    elif choice == "2":
        model_type = "knn"
    else:
        print(" Invalid choice. Exiting.")
        exit()

    # Extract features and predict
    f0, energy = split_audio_pitch("output.wav", sr)
    features = ML_feed(f0, energy)
    emotion = predict_emotion(model_type, features)

    print(f"\n Predicted Emotion: {emotion}")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'emotiondetector_mlp_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)